In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
1,237,SI,VIGENTE,TRÁNSITO RESTRINGIDO,14/01/2023,28/01/2023 10:30 HORAS,MADRE DE DIOS/TAHUAMANU/TAHUAMANU,KM 549,(SECTOR ALERTA) PARO INDEFINIDO SOLICITANDO NU...,-11.665167,...,-11.665167+-69.229412,2023 10:30 HORAS+01+28+10:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
2,250,SI,VIGENTE,TRÁNSITO RESTRINGIDO,06/01/2023,28/01/2023 10:30 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931332,...,-11.931332+-69.11748,2023 10:30 HORAS+01+28+10:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
4,155,SI,VIGENTE,TRÁNSITO RESTRINGIDO,11/01/2023,28/01/2023 10:30 HORAS,CUSCO/LA CONVENCION/SANTA ANA,KM 199,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.873069617394,...,-12.873069617394+-72.692213162262,2023 10:30 HORAS+01+28+10:30,0,0,COE-MTC,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,restringido
5,147,SI,VIGENTE,TRÁNSITO RESTRINGIDO,10/01/2023,28/01/2023 10:30 HORAS,CUSCO/ESPINAR/ESPINAR,KM 245,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-14.794379646982,...,-14.794379646982+-71.409019289269,2023 10:30 HORAS+01+28+10:30,0,0,COE-MTC,HUMANO,PE-3SG,CARRETERA CHALHUAHUACHO - ESPINAR - AYAVIRI,SI,restringido
7,213,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,28/01/2023,28/01/2023 10:30 HORAS,JUNIN/YAULI/MOROCOCHA,KM 140,(ENTRADA A LA MINERA DUVAS) PARO INDEFINIDO SO...,-11.59928542,...,-11.59928542+-76.13262251,2023 10:30 HORAS+01+28+10:30,0,13,SUTRAN UD LIMA BASE MATUCANA,HUMANO,PE-22,CARRETERA CENTRAL,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,350,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,28/01/2023 10:30 HORAS,UCAYALI/PADRE ABAD/IRAZOLA,KM 373,CENTRO POBLADO SAN JUAN BAUTISTA PARO INDEFINI...,-8.904779061,...,-8.904779061+-75.34496538,2023 10:30 HORAS+01+28+10:30,0,0,CEOPOL / UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
88,351,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,28/01/2023 10:30 HORAS,UCAYALI/PADRE ABAD/IRAZOLA,KM 381,CENTRO POBLADO SAN PEDRO DE CHIO PARO INDEFINI...,-8.94135101,...,-8.94135101+-75.38915991,2023 10:30 HORAS+01+28+10:30,0,0,CEOPOL / UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
89,353,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,28/01/2023 10:30 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 388,CENTRO POBLADO HUIPOCA PARO INDEFINIDO SOLICIT...,-8.975618028,...,-8.975618028+-75.42868134,2023 10:30 HORAS+01+28+10:30,0,0,CEOPOL / UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
90,355,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,28/01/2023 10:30 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 405,CENTRO POBLADO BARRIO UNIDO PARO INDEFINIDO SO...,-9.048989785,...,-9.048989785+-75.5106878,2023 10:30 HORAS+01+28+10:30,0,0,CEOPOL / UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
